In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('all stocks50 days.xlsx', index_col=0)

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['Capital Gains'])
df_copy = df_copy.drop(columns=['Adj Close'])

In [ ]:
indexes = df_copy.Symbol.value_counts().loc[lambda x: x>=50].index  #get indexes that have 50 or more entries
df_g50 = df_copy[df_copy['Symbol'].isin(indexes)] #put only those indexes into df

In [ ]:
#this cell runs slow - i will buy you a cookie if you can speed it up

reg = LinearRegression()
df_g50['High slope 20 days'] = 0
df_g50['Close slope 20 days'] = 0
df_g50['High Linear reg MSE'] = 0
df_g50['Close Linear reg MSE'] = 0

X_train = np.arange(0, 20, 1).reshape(-1, 1)
X_test = np.arange(20, 25, 1).reshape(-1, 1)

unique_symbols = df_g50['Symbol'].unique()

#for index in range(5):   #for testing - leave in place
 #   symbol = unique_symbols[index]

#predict on the first 20 and test on the next 5 - in future versions these will be learned params
for symbol in unique_symbols:
    subset = df_g50[df_g50['Symbol'] == symbol].iloc[0:20, :]
    Y1 = subset['High']
    Y2 = subset['Close']
    
    subset = df_g50[df_g50['Symbol'] == symbol].iloc[20:25, :]
    model1 = reg.fit(X_train, Y1)
    prediction1 = model1.predict(X_test)
    mse1 = mean_squared_error(subset['High'], prediction1)
    
    model2 = reg.fit(X_train, Y2)
    prediction2 = model2.predict(X_test)
    mse2 = mean_squared_error(subset['Close'], prediction2)
    
    subset = df_g50[df_g50['Symbol'] == symbol]
    df_g50.loc[subset.index, 'High slope 20 days'] = model1.coef_[0]
    df_g50.loc[subset.index, 'Close slope 20 days'] = model2.coef_[0]
    df_g50.loc[subset.index, 'High Linear reg MSE'] = mse1
    df_g50.loc[subset.index, 'Close Linear reg MSE'] = mse2
    
    
#display(df_g50)

In [ ]:
df_b20 = df_g50.groupby('Symbol').apply(lambda group: group.iloc[20:, :])
stocks_to_buy = df_b20[df_b20['Volume'] > 0]


indexes = stocks_to_buy.Symbol.value_counts().loc[lambda x: x>=30].index  
stocks_to_buy = stocks_to_buy[stocks_to_buy['Symbol'].isin(indexes)] 

In [ ]:
display(stocks_to_buy)
stocks_to_buy.to_csv('stocks to buy.csv', index=True)